<a href="https://colab.research.google.com/github/liminovna/palladizator/blob/colab_version/palladizator_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Добро пожаловать в "Палладизатор"! 

С помощью этой простенькой программы вы можете транскрибировать текст с китайского языка на русский.

Для этого достаточно сделать несколько простых шагов:

1. Первым делом нужно установить необходимые компоненты, нажав на кнопку ▶ в ячейке ниже. Этот шаг нужно выполнить всего один раз.

In [ ]:
!git clone -b colab_version https://github.com/liminovna/palladizator.git palladizator

%cd palladizator

!pip install -r requirements.txt

2. Нажимаем ▶, чтобы импортировать библиотеки и сделать другие важные штуки. Это также достаточно сделать всего один раз.

In [ ]:
#@title
from g2pc import G2pC
import re
import syllable_dict

class zh_phrase:

    is_zh_word = re.compile('([a-z]+)([0-9]+)')

    def __init__(self, phrase, if_capitalize) -> None:
        
        self.capitalize = if_capitalize
        self.phrase = phrase

        self.pinyin_list = self.get_pinyin_list()

    def get_pinyin_list(self) -> list:
        g2p = G2pC()
        return [i[2] for i in g2p(self.phrase.strip())]

    def remove_tones(self) -> list:
        
        dropped_tones = []

        def helper(segment):
            reg_query = re.search(self.is_zh_word, segment)

            if reg_query is not None:
                segment = segment[:reg_query.start()] + segment[reg_query.start():reg_query.end()-1] + segment[reg_query.end():]
                helper(segment)
            else:
                dropped_tones.append(segment)

        for i in range(len(self.pinyin_list)):
            helper(self.pinyin_list[i])

        return dropped_tones

    def get_ru_list(self) -> list:

        palladiused = []
            
        for i in range(len(self.pinyin_list)):
            seg = self.pinyin_list[i]

            new_seg = ''
            if re.match('[a-z]+', seg) is not None:
                for j in re.findall('[a-z]+', seg):
                    seg = j

                    if seg[:2] not in ('zh','sh','ch'):
                        first_letter = seg[0]
                        cur_dict = eval('syllable_dict.'+first_letter)
                    else:
                        first_letters = seg[:2]
                        cur_dict = eval('syllable_dict.'+first_letters)
                    
                    new_seg += cur_dict[seg]
                # palladiused.append(new_seg + ' ')
                palladiused.append(new_seg)

            else:
                palladiused.append(seg)

        return palladiused

    def get_ru_text(self) -> str:

        ru_list = self.get_ru_list()
        # extra_spaces = re.compile('( [，。？！》”）])|([《“（] )')

        if self.capitalize:
            # ru_list = list(map(lambda word: ' ' + word[0].upper()+word[1:], ru_list))
            ru_list = list(map(lambda word: word[0].upper()+word[1:], ru_list))

        
        punct = ''.maketrans(
        {'，': ',', '。': '.', '？': '?', '、': ',', '！': '!', '：': ':', '；': ';', '（': '(', '）': ')', '“': '"',
        '”': '"', '-': '-', '《': '«', '》': '»'}) # this line has been adapted from https://github.com/pantherka
        
        # ru_string = ''.join(ru_list).strip().translate(punct).replace('  ', ' ')
        ru_string = ' '.join(ru_list).translate(punct).strip()
        return ru_string

3. Готово! Теперь вы можете вставить текст, который нужно транскрибировать, и выбрать опцию cpitalize, если хотите, чтобы каждое слово начиналось с большой буквы!

Например, при включенной опции результат для фразы "国家语委现代汉语平衡语料库" будет "Гоцзя Юй Вэй Сяньдай Ханьюй Пинхэн Юйляоку", а при выключенной - "гоцзя юй вэй сяньдай ханьюй пинхэн юйляоку".

In [ ]:
phrase = '' #@param {type:"string"}
capitalize = True #@param {type:"boolean"}

s = zh_phrase(phrase, capitalize)

print(s.get_ru_text())

Спасибо, что воспользовались "Палладизатором"!

При возникновении трудностей пишите на vladaliming @ gmail.com